# Librerías y Configuración

In [26]:
import sympy as sp      # Librería CAS
import numpy as np      # Librería de calculo numérico de matrices
sp.init_printing()      # Print en LateX

# Planteamiento de ecuaciones

### Parámetros

In [27]:
# Eslabón 2
l_2 = 400               # Distancia en mm
theta_2_0 = sp.pi/3     # Angulo inicial 60°
omega_2 = sp.pi/4       # rad/s Velocidad de rotación constante
# Eslabón 3
l_3 = 300               # Distancia en mm
theta_3_0 = sp.pi/6     # Angulo inicial 30°
omega_3 = -sp.pi/2      # rad/s Velocidad de rotación constante

## Parámetros de simulación
max_iter = 100          # Numero de iteraciones
max_error = 0.000001    # Error admisible
steps = 100             # Numero de muestras por segundo
time = 1                # Tiempo de simulación en segundos

### Variables Simbólicas

In [28]:
# Incógnitas (q)
## Eslabón 1
R1_x, R1_y, theta_1 = sp.symbols('R^1_x R^1_y theta^1')
## Eslabón 2
R2_x, R2_y, theta_2 = sp.symbols("R^2_x R^2_y theta^2")
## Eslabón 3
R3_x, R3_y, theta_3 = sp.symbols("R^3_x R^3_y theta^3")
## Vector q (Matriz de una columna)
q = sp.Matrix([R1_x, R1_y, theta_1, R2_x, R2_y, theta_2, R3_x, R3_y, theta_3])

# delta_q
delta_R1_x, delta_R1_y, delta_theta_1 = sp.symbols("\Delta{R^1_x} \Delta{R^1_y} \Delta{\\theta^1}")
delta_R2_x, delta_R2_y, delta_theta_2 = sp.symbols("\Delta{R^2_x} \Delta{R^2_y} \Delta{\\theta^2}")
delta_R3_x, delta_R3_y, delta_theta_3 = sp.symbols("\Delta{R^3_x} \Delta{R^3_y} \Delta{\\theta^3}")

delta_q = sp.Matrix([delta_R1_x, delta_R1_y, delta_theta_1, delta_R2_x, delta_R2_y, delta_theta_2, delta_R3_x,
                     delta_R3_y, delta_theta_3])


# Variable de tiempo
t = sp.symbols("t")
# Imprimir tupla con variables para verificar
q,delta_q,t


⎛⎡ R¹ₓ  ⎤  ⎡ \Delta{R_x}__1  ⎤   ⎞
⎜⎢      ⎥  ⎢                 ⎥   ⎟
⎜⎢R_y__1⎥  ⎢ \Delta{R_y}__1  ⎥   ⎟
⎜⎢      ⎥  ⎢                 ⎥   ⎟
⎜⎢  θ¹  ⎥  ⎢\Delta{\theta__1}⎥   ⎟
⎜⎢      ⎥  ⎢                 ⎥   ⎟
⎜⎢ R²ₓ  ⎥  ⎢ \Delta{R_x}__2  ⎥   ⎟
⎜⎢      ⎥  ⎢                 ⎥   ⎟
⎜⎢R_y__2⎥, ⎢ \Delta{R_y}__2  ⎥, t⎟
⎜⎢      ⎥  ⎢                 ⎥   ⎟
⎜⎢  θ²  ⎥  ⎢\Delta{\theta__2}⎥   ⎟
⎜⎢      ⎥  ⎢                 ⎥   ⎟
⎜⎢ R³ₓ  ⎥  ⎢ \Delta{R_x}__3  ⎥   ⎟
⎜⎢      ⎥  ⎢                 ⎥   ⎟
⎜⎢R_y__3⎥  ⎢ \Delta{R_y}__3  ⎥   ⎟
⎜⎢      ⎥  ⎢                 ⎥   ⎟
⎝⎣  θ³  ⎦  ⎣\Delta{\theta__3}⎦   ⎠

### Ecuaciones de Restricción

In [29]:
# Ecuaciones de Juntas: (igualadas a 0)
## Junta Fija
e1 = R1_x
e2 = R1_y
e3 = theta_1

## Junta R2
e4 = R2_x - ((l_2/2) * sp.cos(theta_2))
e5 = R2_y - ((l_2/2) * sp.sin(theta_2))

## Junta R3
e6 = R2_x + ((l_2/2) * sp.cos(theta_2)) - R3_x + ((l_3/2) * sp.cos(theta_3))
e7 = R2_y + ((l_2/2) * sp.sin(theta_2)) - R3_y + ((l_3/2) * sp.sin(theta_3))

## Ecuaciones de gobierno para ambas juntas
e8 = theta_2 - theta_2_0 - (omega_2*t)
e9 = theta_3 - theta_3_0 - (omega_3*t)

# Imprimir tupla con ecuaciones para verificar
e1, e2, e3, e4, e5, e6, e7, e8, e9

### Matrices de Calculo

In [30]:
# Vector de ecuaciones de restricción
C_qt = sp.Matrix([e1, e2, e3, e4, e5, e6, e7, e8, e9])
# Matriz Jacobiana
J_qt = C_qt.jacobian(q)

# Imprimir tupla con matrices para verificar
C_qt, J_qt

⎛⎡                      R¹ₓ                      ⎤                            
⎜⎢                                               ⎥                            
⎜⎢                    R_y__1                     ⎥  ⎡1  0  0  0  0        0   
⎜⎢                                               ⎥  ⎢                         
⎜⎢                      θ¹                       ⎥  ⎢0  1  0  0  0        0   
⎜⎢                                               ⎥  ⎢                         
⎜⎢              R²ₓ - 200.0⋅cos(θ²)              ⎥  ⎢0  0  1  0  0        0   
⎜⎢                                               ⎥  ⎢                         
⎜⎢            R_y__2 - 200.0⋅sin(θ²)             ⎥  ⎢0  0  0  1  0  200.0⋅sin(
⎜⎢                                               ⎥  ⎢                         
⎜⎢   R²ₓ - R³ₓ + 200.0⋅cos(θ²) + 150.0⋅cos(θ³)   ⎥, ⎢0  0  0  0  1  -200.0⋅cos
⎜⎢                                               ⎥  ⎢                         
⎜⎢R_y__2 - R_y__3 + 200.0⋅sin(θ²) + 150.0⋅sin(θ³)⎥  

### Aplicación de Newton-Raphson

In [31]:

# Valores iniciales
q_values = sp.Matrix([0, 0, 0, 100, 170, theta_2_0, 330, 420, theta_3_0])

# J_qt(t)*deltaQ = C_qt(t)
for step in range(steps
                  ):
    i: int = 0
    current_error: int = 1
    while(i<max_iter and current_error > max_error):
        # Construir lista de valores de q
        q_values_list = [(R1_x,    q_values[0]),
                         (R1_y,    q_values[1]),
                         (theta_1, q_values[2]),
                         (R2_x,    q_values[3]),
                         (R2_y,    q_values[4]),
                         (theta_2, q_values[5]),
                         (R3_x,    q_values[6]),
                         (R3_y,    q_values[7]),
                         (theta_3, q_values[8]),
                         (t,       (time/steps) * step)]

        # Calcular las matrices de coeficientes y Constantes
        coefficients_matrix = J_qt.subs(q_values_list)
        constants_vector = C_qt.subs(q_values_list)
        expression = sp.Eq(coefficients_matrix*q,-constants_vector)
        # Calcular delta
        solution = sp.solve(expression, q)
        delta_q = sp.Matrix([[solution[R1_x]],
                             [solution[R1_y]],
                             [solution[theta_1]],
                             [solution[R2_x]],
                             [solution[R2_y]],
                             [solution[theta_2]],
                             [solution[R3_x]],
                             [solution[R3_y]],
                             [solution[theta_3]]])

        # Calcular nuevo q
        q_values = q_values + delta_q
        # Calcular error máximo
        current_error = coefficients_matrix.norm()
        # current_error = 0
        # for j in delta_q:
        #     if sp.Abs(j) > sp.Abs(current_error):
        #         current_error = sp.Abs(j)
        print("Time: ", (time/steps) * step)
        print("iteration: ", i)
        print("Error: ", current_error)
        i = i + 1

coefficients_matrix, constants_vector, solution, delta_q, q_values


Time:  0.0
iteration:  0
Error:  320.173390524572
Time:  0.0
iteration:  1
Error:  320.173390524572
Time:  0.0
iteration:  2
Error:  320.173390524572
Time:  0.0
iteration:  3
Error:  320.173390524572
Time:  0.0
iteration:  4
Error:  320.173390524572
Time:  0.0
iteration:  5
Error:  320.173390524572
Time:  0.0
iteration:  6
Error:  320.173390524572
Time:  0.0
iteration:  7
Error:  320.173390524572
Time:  0.0
iteration:  8
Error:  320.173390524572
Time:  0.0
iteration:  9
Error:  320.173390524572
Time:  0.0
iteration:  10
Error:  320.173390524572
Time:  0.0
iteration:  11
Error:  320.173390524572
Time:  0.0
iteration:  12
Error:  320.173390524572
Time:  0.0
iteration:  13
Error:  320.173390524572
Time:  0.0
iteration:  14
Error:  320.173390524572
Time:  0.0
iteration:  15
Error:  320.173390524572
Time:  0.0
iteration:  16
Error:  320.173390524572
Time:  0.0
iteration:  17
Error:  320.173390524572
Time:  0.0
iteration:  18
Error:  320.173390524572
Time:  0.0
iteration:  19
Error:  320.173

⎛⎡1  0  0  0  0                  0                  0   0                  0  
⎜⎢                                                                            
⎜⎢0  1  0  0  0                  0                  0   0                  0  
⎜⎢                                                                            
⎜⎢0  0  1  0  0                  0                  0   0                  0  
⎜⎢                                                                            
⎜⎢                        ⎛                    π⎞                             
⎜⎢0  0  0  1  0  200.0⋅sin⎜0.777544181763473 + ─⎟   0   0                  0  
⎜⎢                        ⎝                    3⎠                             
⎜⎢                                                                            
⎜⎢                         ⎛                    π⎞                            
⎜⎢0  0  0  0  1  -200.0⋅cos⎜0.777544181763473 + ─⎟  0   0                  0  
⎜⎢                         ⎝                    3⎠  